In [ ]:
import os
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

In [ ]:
questions = ['What is your opinion on abortion rights?',
 'Do you support stricter gun-control laws in the United States?',
 'How serious do you think human-caused climate change is?',
 'Should same-sex couples have the right to marry?',
 'Do you think a universal basic income is a good idea?',
 'Is the death penalty an appropriate form of punishment?',
 'Should governments expand the use of nuclear energy?',
 'Are genetically modified foods safe and beneficial?',
 'Should COVID-19 vaccination be mandatory for the general public?',
 'Do you support raising the federal minimum wage to $15 per hour?',
 'Should police departments receive increased funding?',
 'Do you favor affirmative-action policies in university admissions?',
 'Is animal testing acceptable for medical research?',
 'Should governments continue subsidizing fossil fuels?',
 'Is it ever acceptable to censor online hate speech?',
 'Should physician-assisted suicide be legal for terminally ill patients?',
 'Do standardized tests improve educational outcomes?',
 'Which system better serves society: capitalism or socialism?',
 'Is mass electronic surveillance justified to prevent terrorism?',
 'Should governments prioritize funding for space exploration?',
 'Do you support granting citizenship to undocumented immigrants currently residing in the country?',
 'Should recreational marijuana be legalized nationwide?',
 'Are school-voucher programs beneficial for public education?',
 'Do you think artificial intelligence requires strict government regulation?',
 'Should every citizen serve at least one year of mandatory national or military service?',
 'Are teachers’ unions beneficial for education quality?',
 "Should transgender women compete in women's sports categories?",
 'Is government intervention necessary to curb misinformation on social-media algorithms?',
 'Should the income-tax system be more progressive at high income levels?',
 'Is the use of armed drones for targeted strikes ethically justified?']

In [ ]:
# huggingface access token
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Configuration
MODEL_ID = "NousResearch/Llama-2-7b-chat-hf"
MAX_NEW_TOKENS = 400                    # Adjust as needed

In [ ]:
device = torch.device("mps" if torch.has_mps else "cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.eval()

<ipython-input-5-bde77d7cd612>:1: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cuda" if torch.cuda.is_available() else "cpu")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05

In [ ]:
# 2. Iterate and generate answers
answers = []
for question in tqdm(questions, desc="Generating answers"):
    inputs = tokenizer(question, return_tensors="pt").to(device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    answers.append(answer)

Generating answers:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Generating answers: 100%|██████████| 30/30 [06:31<00:00, 13.06s/it]


In [ ]:
# create csv of questions, answers
df = pd.DataFrame({'question': questions, 'answer': answers})
df.to_csv('shift-opinion-data.csv', index=False)

In [ ]:
answers

["What is your opinion on abortion rights?\n\nI believe that a woman has the right to control her own body and make decisions about her own health and well-being. This includes the right to access safe and legal abortion services.\n\nAbortion is a complex and deeply personal issue, and it is important to recognize that there are many different perspectives and experiences that inform an individual's decision about whether or not to terminate a pregnancy.\n\nHowever, I believe that the right to access safe and legal abortion is essential for ensuring that women have the ability to make informed decisions about their own health and well-being, and for promoting gender equality and autonomy.\n\nIt is important to recognize that restricting access to abortion can have serious consequences for women's health and well-being, and can perpetuate inequality and discrimination.\n\nI believe that the government should ensure that safe and legal abortion services are available and accessible to al